In [1]:
from sentiment import get_sentiment

In [2]:
text="I am happy with the services provided"


In [4]:
get_sentiment(text, algo="nltk")



0.5719

In [5]:
get_sentiment(text, algo="textblob")


0.8

In [6]:
text="I am frustrated at not being able to reach your executive"


In [7]:
get_sentiment(text, algo="nltk")


-0.5106

In [8]:
get_sentiment(text, algo="textblob")


-0.09999999999999998